In [389]:
from lxml import etree
from lxml import objectify
from xml.etree import cElementTree as ET
import re
import requests

In [390]:
laws = []
class lei:
    def __init__(self):  
        self.tipoDocumento = ""
        self.date = ""
        self.urn = ""
        self.localidade = ""
        self.autoridade = ""
        self.title = []
        self.descricao = []
        self.identifier = ""
        self.subject = []
        
    def print_self(self):
        return(self.tipoDocumento + "*" + self.date + "*" + self.urn + \
               "*" + self.localidade + "*" + self.autoridade + "*" + "*".join(self.title) + \
               "*" + "*".join(self.descricao) + self.identifier +  \
               "*".join(self.subject) + "\n")

In [391]:
# namespace
ns = {
    'srw_dc': 'info:srw/schema/1/dc-schema',
    'dc': 'http://purl.org/dc/elements/1.1/',
    'srw': 'http://www.loc.gov/zing/srw/',
    'xsi': 'http://www.w3.org/2001/XMLSchema'
}

tD = "tipoDocumento"
date = "{http://purl.org/dc/elements/1.1/}date"
urn = "urn"
localidade = "localidade"
autoridade = "autoridade"
title = "{http://purl.org/dc/elements/1.1/}title"
descrip = "{http://purl.org/dc/elements/1.1/}description"
ID = "{http://purl.org/dc/elements/1.1/}identifier"
subject = "{http://purl.org/dc/elements/1.1/}subject"

In [392]:
# Creates a lei object for the current item.  
# Adds each value to it's respective tag

def get_values(x):
    new_law = lei()
    for i in x.iter():
        tag = i.tag
        if tag == tD:
            new_law.tipoDocumento = i.text
        if tag == date:
            new_law.date = i.text
        if tag == urn:
            new_law.urn = i.text
        if tag == localidade:
            new_law.localidade = i.text
        if tag == autoridade:
            new_law.autoridade = i.text
        if tag == title:
            new_law.title.append(i.text)
        if tag == descrip:
            new_law.descricao.append(i.text)
        if tag == subject:
            subjects = [x.strip() for x in re.split('[\,\.]', i.text)]
            unique_subjects = list(set(subjects))
            new_law.subject = unique_subjects
    return new_law

In [393]:
url = "https://www.lexml.gov.br/busca/SRU?operation=searchRetrieve&query=urn+=%22lei+federal%22&maximumRecords=1000&startRecord=1"

req = requests.request('GET', url)
a = req.content
tree = etree.fromstring(a)



# x stands for each XML entry in <srw_dc:dc>
for x in tree.findall(".//srw_dc:dc", namespaces=ns):
    new_law = get_values(x)
    laws.append(new_law)

In [394]:
file = open("f.txt", "w")  
for i in laws:
    file.write(i.print_self())
file.close()